# Hi

In [32]:
# Load dependencies
# import numpy as np
from ipycanvas import Canvas,  hold_canvas
canvas = Canvas(width=800, height=800)

## The First Maze
Leverage the "recursive backtracker" algorithm. It is a randomized version of the depth-first search algorithm.

### The Recursive Algorithm
1. Given a current cell as a parameter, Mark the current cell as visited.
2. While the current cell has any unvisited neighbor cells:
  1. Choose one of the unvisited neighbors.
  2. Remove the wall between the current cell and the chosen cell.
  3. Invoke the routine recursively for a chosen cell.

### The Iterative Algorithm
To avoid a deep recursive, a stack can be used.

1. Choose the initial cell, mark it as visited and push it to the stack.
2. While the stack is not empty:
  1. Pop a cell from the stack and make it a current cell.
  2. If the current cell has any neighbors which have not been visited:
    1. Push the current cell to the stack.
    2. Choose one of the unvisited neighbors.
    3. Remove the wall between the current cell and the chosen cell.
    4. Mark the chosen cell as visited and push it to the stack.

### The Data Structures
A maze is composed of traversable "cells". For my purposes a cell is defined as:
- Fixed size. The cell cannot grow or shrink.
- Fixed in place. Cells don't move.
- All cells are the same size.
- Composed of 4 sides: north, south, east, west.


In [33]:
from typing import List, NamedTuple

"""
Represents a traversable room in a maze.
"""
class MazeCell:
  north:bool = True
  south:bool = True
  east:bool = True
  west:bool = True
  visited:bool = True
  
  def __init__(self) -> None:
    pass

  # TODO: Probably should separate drawing logic from the cell structure.
  # Don't want Cell to know about IpyCanvas
  def draw() -> None:
    pass

class Corner(NamedTuple):
  x: int
  y: int

# Typing alias for the maze.
Maze = List[List[MazeCell]]

"""
Builds a rectangular grid of cells were all the walls are intially closed.
"""
def pack_grid(width: int, height: int) -> Maze:
  maze: Maze = []
  for h in range(height):
    row: list[MazeCell] = []
    for h in range(width):
      row.append(MazeCell())
    maze.append(row)
  return maze

"""
Traverses the grid of cells creates a maze by opening walls.
"""
def generate_maze_walls(maze: Maze) -> Maze:
  pass

def draw_wall(canvas: Canvas, start: Corner, stop: Corner) -> None:
  canvas.move_to(start.x, start.y)
  canvas.line_to(stop.x, stop.y)
  canvas.stroke()

def draw_cell_walls(cell: MazeCell, canvas: Canvas, cell_index: int, row_index: int, room_size_width: int, room_size_height: int) -> None:
  canvas.begin_path()

  upper_left_corner = Corner(cell_index*room_size_width, row_index*room_size_height)
  upper_right_corner = Corner(upper_left_corner.x + room_size_width, upper_left_corner.y)
  lower_right_corner = Corner(upper_right_corner.x, upper_right_corner.y + room_size_height)
  lower_left_corner = Corner(lower_right_corner.x - room_size_width, lower_right_corner.y)

  if cell.north: draw_wall(canvas, upper_left_corner, upper_right_corner)
  if cell.east: draw_wall(canvas, upper_right_corner, lower_right_corner)
  if cell.south: draw_wall(canvas, lower_right_corner, lower_left_corner)
  if cell.west: draw_wall(canvas, lower_left_corner, upper_left_corner)
  
  return

"""Draws the maze."""
def draw_maze(maze: Maze, canvas: Canvas) -> Canvas:
  room_size_width: int =  20
  room_size_height: int = 20
  
  with hold_canvas(canvas):
    canvas.line_width = 5
    for row_index in range(len(maze)):
      for cell_index in range(len(maze[row_index])):
        cell = maze[row_index][cell_index]
        draw_cell_walls(cell, canvas, cell_index, row_index, room_size_width, room_size_height)
  return canvas;

In [34]:
maze: Maze = pack_grid(40, 40)
generate_maze_walls(maze)
canvas = draw_maze(maze, canvas)
display(canvas)

Canvas(height=800, width=800)